In [2]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2

from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.applications.vgg16 import  VGG16
import os
import seaborn as sns


In [3]:
print(os.listdir("./dataset/"))

['train2', 'valid2', 'test']


In [4]:
SIZE = 256 # Resized Image

In [5]:
train_images = []
train_labels = []

- Using directory_path.split("\\")\
- Ouput:\
['./dataset/train2', 'cats']\
['./dataset/train2', 'dogs']
- glob return ./dataset/train2\cats

In [16]:
for directory_path in glob.glob("./dataset/train2/*"):
    label = directory_path.split("\\")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR) # cv2.imread_color is default of cv2.imread() function
        if img is not None:
            img = cv2.resize(img, (SIZE, SIZE))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            train_images.append(img)
            train_labels.append(label)

# Convert lists to arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [ ]:
# Captures test/validation data and labels into respective lists
